In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [20]:
data = pd.read_csv('data_final.csv')
# Keeping only the neccessary columns
data = data[['Headline','Price movement']]

In [21]:
data = data[data['Price movement'] != "#N/A"]
data['Headline'] = data['Headline'].apply(lambda x: x.lower())
data['Headline'] = data['Headline'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[data['Price movement'] == 'Up'].size)
print(data[data['Price movement'] == 'Down'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Headline'].values)
X = tokenizer.texts_to_sequences(data['Headline'].values)
X = pad_sequences(X)

6040
4916


In [22]:
from keras.layers import Conv1D, MaxPooling1D, Flatten

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 35, 128)           256000    
                                                                 
 conv1d_9 (Conv1D)           (None, 35, 128)           65664     
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 17, 128)          0         
 1D)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 196)               254800    
                                                                 
 flatten_3 (Flatten)         (None, 196)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               50432     
                                                      

In [23]:
Y = pd.get_dummies(data['Price movement']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3778, 35) (3778, 2)
(1862, 35) (1862, 2)


In [24]:
batch_size = 32
model.fit(X_train, Y_train, epochs=20, batch_size=batch_size, verbose = 2)

Epoch 1/20
119/119 - 6s - loss: 0.6894 - accuracy: 0.5159 - 6s/epoch - 50ms/step
Epoch 2/20
119/119 - 4s - loss: 0.7383 - accuracy: 0.5199 - 4s/epoch - 33ms/step
Epoch 3/20
119/119 - 4s - loss: 0.7326 - accuracy: 0.5019 - 4s/epoch - 32ms/step
Epoch 4/20
119/119 - 4s - loss: 0.6995 - accuracy: 0.5196 - 4s/epoch - 31ms/step
Epoch 5/20
119/119 - 4s - loss: 0.8173 - accuracy: 0.4894 - 4s/epoch - 30ms/step
Epoch 6/20
119/119 - 4s - loss: 0.8235 - accuracy: 0.4974 - 4s/epoch - 30ms/step
Epoch 7/20
119/119 - 4s - loss: 0.8858 - accuracy: 0.5058 - 4s/epoch - 33ms/step
Epoch 8/20
119/119 - 4s - loss: 0.7644 - accuracy: 0.5053 - 4s/epoch - 34ms/step
Epoch 9/20
119/119 - 4s - loss: 1.0074 - accuracy: 0.5032 - 4s/epoch - 32ms/step
Epoch 10/20
119/119 - 4s - loss: 0.8529 - accuracy: 0.5127 - 4s/epoch - 32ms/step
Epoch 11/20
119/119 - 4s - loss: 0.8244 - accuracy: 0.5032 - 4s/epoch - 36ms/step
Epoch 12/20
119/119 - 4s - loss: 0.8410 - accuracy: 0.5101 - 4s/epoch - 31ms/step
Epoch 13/20
119/119 - 4s 

In [25]:
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=2)
print("Accuracy: %.2f%%" % (scores[1]*100))

59/59 - 1s - loss: 1.0325 - accuracy: 0.4796 - 606ms/epoch - 10ms/step
Accuracy: 47.96%
